# 12. Summarizing and Aggregating

In [ ]:
import polars as pl
pl.__version__

'1.20.0'

## Split, Apply, and Combine

## GroupBy Context

In [2]:
fruit = pl.read_csv("data/fruit.csv")
fruit_grouped = fruit.group_by("is_round")
fruit_grouped

In [3]:
fruit_grouped.len()

is_round,len
bool,u32
false,6
true,4


In [4]:
top2000 = pl.read_excel(
    "data/top2000-2023.xlsx", read_options={"skip_rows": 1}
).set_sorted("positie")

In [5]:
(
    top2000.group_by("jaar")
    .agg(
        songs=pl.concat_str(
            pl.col("artiest"), pl.lit(" - "), pl.col("titel")
        ),
    )
    .sort("jaar", descending=True)
)

jaar,songs
i64,list[str]
2022,"[""Son Mieux - Multicolor"", ""Bankzitters - Je Blik Richting Mij"", … ""Måneskin - THE LONELIEST""]"
2021,"[""Goldband - Noodgeval"", ""Bankzitters - Stapelgek"", … ""Olivia Rodrigo - Drivers License""]"
2020,"[""DI-RECT - Soldier On"", ""Miss Montreal - Door De Wind"", … ""Dua Lipa ft. DaBaby - Levitating""]"
2019,"[""Danny Vera - Roller Coaster"", ""Floor Jansen & Henk Poort - Phantom Of The Opera"", … ""Tino Martin - Zij Weet Het""]"
2018,"[""Lady Gaga & Bradley Cooper - Shallow"", ""White Lies - Time To Give"", … ""Calvin Harris & Dua Lipa - One Kiss""]"
…,…
1960,"[""Etta James - At Last"", ""Shadows - Apache""]"
1959,"[""Jacques Brel - Ne Me Quitte Pas"", ""Elvis Presley - Hound Dog""]"
1958,"[""Chuck Berry - Johnny B. Goode"", ""Ella Fitzgerald & Louis Armstrong - Summertime""]"


### The Descriptives

In [6]:
(
    top2000.group_by("jaar", maintain_order=True)
    .head(3)
    .sort("jaar", descending=True)
    .head(9)
)

jaar,positie,titel,artiest
i64,i64,str,str
2022,179,"""Multicolor""","""Son Mieux"""
2022,370,"""Je Blik Richting Mij""","""Bankzitters"""
2022,395,"""L'enfer""","""Stromae"""
2021,55,"""Noodgeval""","""Goldband"""
2021,149,"""Stapelgek""","""Bankzitters"""
2021,210,"""Dat Heb Jij Gedaan""","""Meau"""
2020,19,"""Soldier On""","""DI-RECT"""
2020,38,"""Door De Wind""","""Miss Montreal"""
2020,77,"""Impossible (Orchestral Version…","""Nothing But Thieves"""


In [7]:
(
    top2000.group_by("jaar", maintain_order=True)
    .tail(3)
    .sort("jaar", descending=True)
    .head(9)
)

jaar,positie,titel,artiest
i64,i64,str,str
2022,1391,"""De Diepte""","""S10"""
2022,1688,"""Zeit""","""Rammstein"""
2022,1716,"""THE LONELIEST""","""Måneskin"""
2021,1865,"""Bon Gepakt""","""Donnie & Rene Froger"""
2021,1978,"""Hold On""","""Armin van Buuren ft. Davina Mi…"
2021,2000,"""Drivers License""","""Olivia Rodrigo"""
2020,1824,"""Smoorverliefd""","""Snelle"""
2020,1879,"""The Business""","""Tiësto"""
2020,1902,"""Levitating""","""Dua Lipa ft. DaBaby"""


In [8]:
(top2000.group_by("artiest").len().sort("len", descending=True).head(10))

artiest,len
str,u32
"""Queen""",34
"""The Beatles""",31
"""ABBA""",25
"""Bruce Springsteen""",22
"""The Rolling Stones""",22
"""Michael Jackson""",20
"""Coldplay""",20
"""Fleetwood Mac""",20
"""U2""",18


In [9]:
sales = pl.read_csv("data/sales.csv")
sales.columns

['Date',
 'Day',
 'Month',
 'Year',
 'Customer_Age',
 'Age_Group',
 'Customer_Gender',
 'Country',
 'State',
 'Product_Category',
 'Sub_Category',
 'Product',
 'Order_Quantity',
 'Unit_Cost',
 'Unit_Price',
 'Profit',
 'Cost',
 'Revenue']

In [10]:
(
    sales.select("Product_Category", "Sub_Category", "Unit_Price")
    .group_by("Product_Category", "Sub_Category")
    .max()
    .sort("Unit_Price", descending=True)
    .head(10)
)

Product_Category,Sub_Category,Unit_Price
str,str,i64
"""Bikes""","""Road Bikes""",3578
"""Bikes""","""Mountain Bikes""",3400
"""Clothing""","""Vests""",2384
"""Bikes""","""Touring Bikes""",2384
"""Accessories""","""Bike Stands""",159
"""Accessories""","""Bike Racks""",120
"""Clothing""","""Shorts""",70
"""Clothing""","""Socks""",70
"""Accessories""","""Hydration Packs""",55


In [11]:
(
    sales.select("Country", "Profit")
    .group_by("Country")
    .sum()
    .sort("Profit", descending=True)
)

Country,Profit
str,i64
"""United States""",11073644
"""Australia""",6776030
"""United Kingdom""",4413853
"""Canada""",3717296
"""Germany""",3359995
"""France""",2880282


In [12]:
(
    sales.select("Sub_Category", "Product")
    .group_by("Sub_Category")
    .n_unique()
    .sort("Product", descending=True)
    .head(10)
)

Sub_Category,Product
str,u32
"""Road Bikes""",38
"""Mountain Bikes""",28
"""Touring Bikes""",22
"""Tires and Tubes""",11
"""Jerseys""",8
"""Gloves""",4
"""Vests""",4
"""Socks""",3
"""Shorts""",3


In [13]:
(
    sales.select("Age_Group", "Order_Quantity")
    .group_by("Age_Group")
    .mean()
    .sort("Order_Quantity", descending=True)
)

Age_Group,Order_Quantity
str,f64
"""Seniors (64+)""",13.530137
"""Youth (<25)""",12.124018
"""Adults (35-64)""",12.045303
"""Young Adults (25-34)""",11.560899


In [14]:
(
    sales.select("Age_Group", "Revenue")
    .group_by("Age_Group")
    .quantile(0.9)
    .sort("Revenue", descending=True)
)

Age_Group,Revenue
str,f64
"""Young Adults (25-34)""",2227.0
"""Adults (35-64)""",2217.0
"""Youth (<25)""",1997.0
"""Seniors (64+)""",943.0


### Advanced Methods

#### Aggregate values to a List

In [15]:
(
    sales.select("Country", "Profit", "Revenue")
    .group_by("Country")
    .agg(
        pl.col("Profit"),
        pl.col("Revenue"),
    )
)

Country,Profit,Revenue
str,list[i64],list[i64]
"""Germany""","[160, 53, … 746]","[295, 98, … 1250]"
"""Canada""","[590, 590, … 630]","[950, 950, … 1014]"
"""France""","[427, 427, … 655]","[787, 787, … 1207]"
"""United Kingdom""","[1053, 1053, … 112]","[1728, 1728, … 184]"
"""United States""","[524, 407, … 542]","[929, 722, … 878]"
"""Australia""","[1366, 1188, … 655]","[2401, 2088, … 1183]"


#### Rename aggregated columns

In [16]:
(
    sales.group_by("Country").agg(
        pl.col("Profit").alias("All Profits Per Transactions"),
        pl.col("Revenue").name.prefix("All "),
        Cost=pl.col("Revenue") - pl.col("Profit"),
    )
)

Country,All Profits Per Transactions,All Revenue,Cost
str,list[i64],list[i64],list[i64]
"""France""","[427, 427, … 655]","[787, 787, … 1207]","[360, 360, … 552]"
"""United Kingdom""","[1053, 1053, … 112]","[1728, 1728, … 184]","[675, 675, … 72]"
"""Australia""","[1366, 1188, … 655]","[2401, 2088, … 1183]","[1035, 900, … 528]"
"""Germany""","[160, 53, … 746]","[295, 98, … 1250]","[135, 45, … 504]"
"""United States""","[524, 407, … 542]","[929, 722, … 878]","[405, 315, … 336]"
"""Canada""","[590, 590, … 630]","[950, 950, … 1014]","[360, 360, … 384]"


#### Apply multiple aggregations at once

In [17]:
(
    sales.select("Country", "Profit", "Revenue")
    .group_by("Country")
    .agg(
        pl.col("Profit").sum().name.prefix("Total "),
        pl.col("Profit").mean().alias("Average Profit per Transaction"),
        pl.col("Revenue").sum().name.prefix("Total "),
        pl.col("Revenue").mean().alias("Average Revenue per Transaction"),
    )
)

Country,Total Profit,Average Profit per Transaction,Total Revenue,Average Revenue per Transaction
str,i64,f64,i64,f64
"""Canada""",3717296,262.187615,7935738,559.721964
"""France""",2880282,261.891435,8432872,766.764139
"""United States""",11073644,282.447687,27975547,713.552696
"""Australia""",6776030,283.089489,21302059,889.959016
"""United Kingdom""",4413853,324.071439,10646196,781.659031
"""Germany""",3359995,302.756803,8978596,809.028293


In [18]:
(
    sales.select("Country", "Profit", "Revenue")
    .group_by("Country")
    .agg(
        pl.all().sum().name.prefix("Total "),
        pl.all().mean().name.prefix("Average "),
    )
)

Country,Total Profit,Total Revenue,Average Profit,Average Revenue
str,i64,i64,f64,f64
"""France""",2880282,8432872,261.891435,766.764139
"""United States""",11073644,27975547,282.447687,713.552696
"""Canada""",3717296,7935738,262.187615,559.721964
"""United Kingdom""",4413853,10646196,324.071439,781.659031
"""Germany""",3359995,8978596,302.756803,809.028293
"""Australia""",6776030,21302059,283.089489,889.959016


In [19]:
(
    sales.select("Country", "Profit")
    .group_by("Country")
    .agg(
        (pl.col("Profit") > 1000).alias("Profit > 1000"),
        (pl.col("Profit") > 1000)
        .sum()
        .alias("Transactions with Profit > 1000"),
    )
)

Country,Profit > 1000,Transactions with Profit > 1000
str,list[bool],u32
"""United Kingdom""","[true, true, … false]",788
"""Canada""","[false, false, … false]",868
"""France""","[false, false, … false]",482
"""United States""","[false, false, … false]",2623
"""Germany""","[false, false, … false]",659
"""Australia""","[true, true, … false]",1233


In [20]:
def sum_transactions_above_threshold(
    col: pl.Expr, threshold: float
) -> tuple[pl.Expr, pl.Expr]:
    """Sums transactions where the column col exceeds specified threshold"""
    original_column_name = col.meta.root_names()[0]
    condition_column = (col > threshold).alias(
        f"{original_column_name} > {threshold}"
    )
    new_column = (
        (col > threshold)
        .sum()
        .alias(f"Transactions with {original_column_name} > {threshold}")
    )
    return condition_column, new_column


sales.select("Country", "Profit").group_by("Country").agg(
    sum_transactions_above_threshold(pl.col("Profit"), 999)
)

Country,Profit > 999,Transactions with Profit > 999
str,list[bool],u32
"""United States""","[false, false, … false]",2623
"""Germany""","[false, false, … false]",659
"""France""","[false, false, … false]",482
"""Australia""","[true, true, … false]",1233
"""Canada""","[false, false, … false]",868
"""United Kingdom""","[true, true, … false]",788


## Row-Wise Aggregations

In [21]:
fold_example = pl.DataFrame({"col1": [2], "col2": [3], "col3": [4]})

fold_example.with_columns(
    sum=pl.fold(
        acc=pl.lit(0),
        function=lambda acc, x: acc + x,
        exprs=pl.col("*"),
    )
)

col1,col2,col3,sum
i64,i64,i64,i64
2,3,4,9


In [22]:
products = pl.DataFrame(
    {
        "product_A": [10, 20, 30],
        "product_B": [20, 30, 40],
        "product_C": [30, 40, 50],
    }
)

weights = {"product_A": 0.5, "product_B": 1.5, "product_C": 2.0}

weighted_exprs = [
    (pl.col(product) * weight).alias(product)
    for product, weight in weights.items()
]

products_with_weighted_sum = products.with_columns(
    weighted_sum=pl.fold(
        acc=pl.lit(0),
        function=lambda acc, x: acc + x,
        exprs=weighted_exprs,
    )
)

products_with_weighted_sum

product_A,product_B,product_C,weighted_sum
i64,i64,i64,f64
10,20,30,95.0
20,30,40,135.0
30,40,50,175.0


## Window Functions in Selection Context

In [23]:
(
    top2000.select(
        "jaar",
        "artiest",
        "titel",
        "positie",
        year_rank=pl.col("positie").rank().over("jaar"),
    ).sample(10, seed=42)
)

jaar,artiest,titel,positie,year_rank
i64,str,str,i64,f64
2013,"""Stromae""","""Papaoutai""",318,6.0
1969,"""John Denver""","""Leaving On A Jet Plane""",607,16.0
1971,"""Led Zeppelin""","""Immigrant Song""",590,19.0
2009,"""Anouk""","""For Bitter Or Worse""",1453,23.0
2015,"""Snollebollekes""","""Links Rechts""",1076,14.0
1984,"""Alphaville""","""Forever Young""",302,11.0
1977,"""ABBA""","""Take A Chance On Me""",636,23.0
1975,"""Rod Stewart""","""Sailing""",918,20.0
1986,"""Metallica""","""Master Of Puppets""",29,1.0


## Dynamic Grouping

## Rolling Aggregations

In [24]:
dates = pl.date_range(
    start=pl.date(2024, 4, 1),
    end=pl.date(2024, 4, 26),
    interval="2d",
    eager=True,
)
dates = dates.filter(dates.dt.weekday() < 6)
dates_repeated = pl.concat([dates, dates]).sort()

small_sales_df = (
    pl.DataFrame(
        {
            "date": dates_repeated,
            "store": ["Store A", "Store B"] * dates.len(),
            "sales": [
                200, 150, 220, 160, 250, 180, 270, 190, 280, 210,
                210, 170, 220, 180, 240, 190, 250, 200, 260, 210,
            ],
        }
    )
    .set_sorted("date")
    .set_sorted("store")
)

In [26]:
result = small_sales_df.rolling(
    index_column="date",
    period="7d",
    group_by="store",
).agg(
    sum_of_last_7_days_sales=pl.sum("sales")
)

final_df = small_sales_df.join(result, on=["date", "store"])

final_df

date,store,sales,sum_of_last_7_days_sales
date,str,i64,i64
2024-04-01,"""Store A""",200,200
2024-04-03,"""Store A""",220,420
2024-04-05,"""Store A""",250,670
2024-04-09,"""Store A""",270,740
2024-04-11,"""Store A""",280,800
…,…,…,…
2024-04-15,"""Store B""",170,570
2024-04-17,"""Store B""",180,560
2024-04-19,"""Store B""",190,540


## Upsampling

In [25]:
upsampled_small_sales_df = small_sales_df.upsample(
    time_column="date", every="1d", group_by="store", maintain_order=True
)

upsampled_small_sales_df

date,store,sales
date,str,i64
2024-04-01,"""Store A""",200
2024-04-02,null,null
2024-04-03,"""Store A""",220
2024-04-04,null,null
2024-04-05,"""Store A""",250
…,…,…
2024-04-21,null,null
2024-04-22,null,null
2024-04-23,"""Store B""",200


In [27]:
upsampled_small_sales_df.select(
    "date", pl.col("store").forward_fill(), pl.col("sales").interpolate()
)

date,store,sales
date,str,f64
2024-04-01,"""Store A""",200.0
2024-04-02,"""Store A""",210.0
2024-04-03,"""Store A""",220.0
2024-04-04,"""Store A""",235.0
2024-04-05,"""Store A""",250.0
…,…,…
2024-04-21,"""Store B""",195.0
2024-04-22,"""Store B""",197.5
2024-04-23,"""Store B""",200.0
